In [3]:
import numpy as np
import matplotlib.pyplot as plt

class GaussianRegressionWithDerivatives():
    def __init__(self, kernel=None, dd_kernel=None, sigma=1e-6):
        self.kernel = kernel
        self.dd_kernel = dd_kernel
        self.sigma = sigma
        self.X = None
        self.y = None
        
    def fit(self, X, y, dy):
        self.X = X
        self.y = y
        self.dy = dy
        # print('the shape of self.kernel(X, X) is: ' + str(self.kernel(X, X).shape))
        self.K = self.kernel(X, X) + self.sigma**2 * np.eye(len(X))
        # print('the shape of self.K is: ' + str(self.K.shape))
        self.dK = self.dd_kernel(X, X) + self.sigma**2 * np.eye(len(X))
        self.L = np.linalg.cholesky(self.K)
        self.dL = np.linalg.cholesky(self.dK)
        # let alpha = inv(K) * y => K * alpha = y => L*L.T * alpha = y => alpha = solve(L.T, solve(L, y))
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, y))
        self.dalpha = np.linalg.solve(self.dL.T, np.linalg.solve(self.dL, dy))

        return self
        
    def predict(self, Xtest):

        K_train_test = self.kernel(self.X, Xtest)
        y_pred = K_train_test.T @ self.alpha

        dK_train_test = self.dd_kernel(self.X, Xtest)
        dy_pred = dK_train_test.T @ self.dalpha

        # L*v = K => v = inv(L)*K => v.T * v = K.T * inv(L.T) * inv(L) * K = K.T * inv(L*L.T) * K
        # v = cp.linalg.solve(self.L, K_train_test)
        # cov = self.kernel(Xtest, Xtest) - v.T @ v
        # std = cp.sqrt(cp.diag(cov))

        # return mean, cov, std
        
        return y_pred, dy_pred

def d_mu(x):
    return 0 

def RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return np.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def d_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return diff_mat * cp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)

def dd_RBF(x1, x2, length_scale=1.0):
    diff_mat = x1[:, cp.newaxis] - x2
    return (1 - diff_mat**2/length_scale**2)/length_scale**2 * cp.exp(-0.5 * diff_mat ** 2 / length_scale ** 2)


X = cp.array(cp.random.uniform(-5, 5, size=(50,)))
y = cp.array(cp.sin(X) + cp.random.normal(0, 0.1, size=(50,)))
dy = cp.array(cp.cos(X))

gr = GaussianRegressionWithDerivatives(RBF, dd_RBF)
gr.fit(X,y,dy)

# use the same data set as training, y_pred and dy_pred should be similar to y and dy
y_pred, dy_pred = gr.predict(X)

print('y is: \n' + str(y[:10]))
print('y_pred is: \n' + str(y_pred[:10]))
print('dy is: \n' + str(dy[:10]))
print('dy_pred: \n' + str(dy_pred[:10]))


ModuleNotFoundError: No module named 'cupy'